In [1]:
%cd ..
%pwd
import sys
sys.path.append('./src')
from src.data_processor import DataProcessor
from src.problem_config import create_prob_config 
prob_config = create_prob_config("phase-2", "prob-2")

c:\VENV\api_prediction\monitor_train


## Load Training Data

In [15]:
import pandas as pd
import pickle

training_data = pd.read_parquet(prob_config.raw_data_path)

training_data, category_index = DataProcessor.build_category_features(
            training_data, prob_config.categorical_cols
        )

target_col = prob_config.target_col
train_x = training_data.drop([target_col], axis=1)
train_y = training_data[[target_col]]

# Store the category_index
with open(prob_config.category_index_path, "wb") as f:
    pickle.dump(category_index, f)

# Check and process saved request

In [6]:
%ls "data"

 Volume in drive C is Windows
 Volume Serial Number is 788F-AEF3

 Directory of c:\VENV\api_prediction\monitor_train\data

07/10/2023  10:33 AM    <DIR>          .
07/10/2023  10:25 AM    <DIR>          ..
07/10/2023  10:32 AM        67,043,899 192.168.88.113_prob1_10_07_2023__10_32_28.pkl
07/10/2023  10:33 AM        66,963,641 192.168.88.113_prob2_10_07_2023__10_33_13.pkl
               2 File(s)    134,007,540 bytes
               2 Dir(s)  80,110,854,144 bytes free


In [7]:
saved_request = DataProcessor.load_saved_request('192.168.88.113_prob2_10_07_2023__10_33_13.pkl')
keys = list(saved_request.keys())

# Test API

In [8]:
rows = []
test = saved_request[keys[8]]

for index, row in test.iterrows():
    rows.append(row.to_list())

data = {
  "id": 123,
  "rows": rows,
  "columns": test.columns.to_list()
}

import requests
#response = requests.post('http://14.225.205.204:80/phase-2/prob-2/predict', json=data)
#response = requests.post('http://27.71.25.203:5040/phase-2/prob-2/predict', json=data)
response = requests.post('http://localhost:8000/phase-2/prob-2/predict', json=data)
print(response.text)

{"id":"123","predictions":["Normal","Information Gathering","Normal","Exploits","Exploits","Exploits","Normal","Other","Information Gathering","Exploits","Normal","Information Gathering","Normal","Normal","Information Gathering","Normal","Normal","Normal","Other","Exploits","Denial of Service","Normal","Normal","Normal","Normal","Exploits","Other","Other","Normal","Denial of Service","Denial of Service","Denial of Service","Other","Normal","Other","Normal","Other","Information Gathering","Other","Normal","Normal","Normal","Exploits","Other","Normal","Exploits","Normal","Exploits","Normal","Information Gathering","Normal","Normal","Exploits","Exploits","Denial of Service","Normal","Denial of Service","Normal","Normal","Exploits","Normal","Normal","Denial of Service","Exploits","Exploits","Other","Other","Normal","Other","Normal","Normal","Information Gathering","Normal","Exploits","Exploits","Denial of Service","Normal","Exploits","Exploits","Normal","Exploits","Denial of Service","Expl

In [16]:
import random
for i in range(10):
    key = random.randint(0, len(keys))
    id = random.randint(1000, 2000)
    test = saved_request[keys[key]]
    rows = []
    for index, row in test.iterrows():
        rows.append(row.to_list())
    data = {
        "id": id,
        "rows": rows,
        "columns": test.columns.to_list()
        }
    #response = requests.post('http://14.225.205.204:80/phase-2/prob-2/predict', json=data)
    response = requests.post('http://localhost:8000/phase-2/prob-2/predict', json=data)
    #print(response.text)

In [ ]:
from aiocache import Cache
from aiocache.serializers import PickleSerializer

cacherequest = Cache(Cache.REDIS, endpoint="localhost", port=6379, db=1, serializer=PickleSerializer())
await cacherequest.clear()

# Load request data to DF

In [10]:
captured_x = pd.concat(saved_request.values(), ignore_index=True)
captured_x.drop_duplicates(inplace=True, ignore_index=True)

In [22]:
from src.utils import AppPath

In [27]:
import glob
import os
[os.path.basename(x) for x in glob.glob(str(AppPath.MODEL_CONFIG_DIR / '*.yaml'))]

['phase-2_prob-1.yaml', 'phase-2_prob-2.yaml']

['src\\dashboard.py',
 'src\\data_processor.py',
 'src\\problem_config.py',
 'src\\train_model.py',
 'src\\utils.py']